In [305]:
import numpy as np  
import pandas as pd
from pandas import Series, DataFrame
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [306]:
Data=pd.read_csv('R_Input.csv')
Data['dprio0']=Data['dprio']**0
Data['dprio2']=Data['dprio']**2
Data['dprio3']=Data['dprio']**3
Data=Data.rename(columns={'dprio': 'dprio1'})

direction=pd.get_dummies(Data['direction'],drop_first=True)
direction.columns=['direction:'+x for x in direction.columns]

whichholiday_left_LUX=pd.get_dummies(Data['whichholiday_left_LUX'],drop_first=True)
whichholiday_left_LUX.columns=['whichholiday_left_LUX:'+x for x in whichholiday_left_LUX.columns]

whichholiday_right_LUX=pd.get_dummies(Data['whichholiday_right_LUX'],drop_first=True)
whichholiday_right_LUX.columns=['whichholiday_right_LUX:'+x for x in whichholiday_right_LUX.columns]

Data=pd.concat([Data,direction,whichholiday_left_LUX,whichholiday_right_LUX],axis=1)

Data.index=Data['calendar_date']+'_'+Data['direction']

In [307]:
dprio=Data[['dprio'+str(i) for i in range(4)]].drop_duplicates().transpose()
dprio.columns=range(365,-1,-1)

X_data=Data.drop_duplicates(subset=['calendar_date','direction'])[[x for x in Data.columns if 'direction:' in x]\
+[x for x in Data.columns if 'whichholiday_left_LUX:' in x]\
+[x for x in Data.columns if 'whichholiday_right_LUX:' in x]\
+['holiday_from_left_LUX','holiday_from_right_LUX','holiday_to_left_LUX','holiday_to_right_LUX']\
+['pax_total']]

Data['concat']=Data['calendar_date']+'_'+Data['direction']
y_data=Data[['concat','dprio1','pax_total']].pivot(index='concat',columns='dprio1',values='pax_total')

In [308]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=1)

In [298]:
# X_train=X.loc[[x for x in X.index if '2019' not in x],:].values
# X_test=X.loc[[x for x in X.index if '2019' in x],:].values
# y_train=y.loc[[x for x in y.index if '2019' not in x],:].values
# y_test=y.loc[[x for x in y.index if '2019' in x],:].values

In [299]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [300]:
n_input=X_train.shape[1]
n_hidden_1=n_input*2
n_hidden_2=4
n_output=y_train.shape[1]

In [301]:
#dprio=tf.convert_to_tensor(dprio)

In [302]:
# Parameters
learning_rate = 0.01
training_epochs = 100


Z=tf.constant(dprio.values, dtype=tf.float32)
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32)

# Dictionary of Weights and Biases
weights = {
  'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
  'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
  'out': tf.Variable(tf.random_normal([n_hidden_2, n_output]))
}

biases = {
  'b1': tf.Variable(tf.random_normal([n_hidden_1])),
  'b2': tf.Variable(tf.random_normal([n_hidden_2]))
}

# Model Forwar§d Propagation step
def forward_propagation(x):
    # Hidden layer1
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)

    # Output fully connected layer
    out_layer = tf.matmul(layer_2, Z) 
    return out_layer


# Model Outputs
yhat = forward_propagation(X)
ypredict = tf.argmax(yhat, axis=1)

In [303]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train_op = optimizer.minimize(cost)

In [304]:
# Initializing the variables
init = tf.global_variables_initializer()

from datetime import datetime
startTime = datetime.now()

with tf.Session() as sess:
    sess.run(init)
    
    #writer.add_graph(sess.graph)
    #EPOCHS
    for epoch in range(training_epochs):
        #Stochasting Gradient Descent
        for i in range(len(X_train)):
            summary = sess.run(train_op, feed_dict={X: X_train.loc[i: i + 1,:], y: y_train.loc[i:i + 1,:]})
        
        train_accuracy = np.mean(np.argmax(y_train, axis=1) == sess.run(ypredict, feed_dict={X: X_train, y: y_train}))
        test_accuracy  = np.mean(np.argmax(y_test, axis=1) == sess.run(ypredict, feed_dict={X: X_test, y: y_test}))
                
        print("Epoch = %d, train accuracy = %.2f%%, test accuracy = %.2f%%" % (epoch + 1, 100. * train_accuracy, 100. * test_accuracy))
        #print("Epoch = %d, train accuracy = %.2f%%" % (epoch + 1, 100. * train_accuracy))

    sess.close()
print("Time taken:", datetime.now() - startTime)

TypeError: 'DataFrame' objects are mutable, thus they cannot be hashed

In [ ]:
Data['concat']=Data['calendar_date']+'_'+Data['direction']

In [ ]:
                                                                                         
##########################################

X_cluster=Data.drop_duplicates(subset=['calendar_date','direction'])[['direction','whichholiday_left_LUX'
,'whichholiday_right_LUX','holiday_from_left_LUX','holiday_from_right_LUX','holiday_to_left_LUX','holiday_to_right_LUX']]
#.as_matrix()
X_cluster.index=y.index

##########################################

X_prio=Data.drop_duplicates(subset=['calendar_date','direction'])[['direction','whichholiday_left_LUX'
,'whichholiday_right_LUX','holiday_from_left_LUX','holiday_from_right_LUX','holiday_to_left_LUX','holiday_to_right_LUX']]

In [ ]:
X_cluster

In [ ]:


############################

X_dprio=Data['dprio'].unique()
X_dprio2=[x**2 for x in Data['dprio'].unique()]
X_dprio3=[x**3 for x in Data['dprio'].unique()]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_cluster,y, test_size=.2)

In [ ]:
train_test_split(Data, mnist.target, test_size=0.2, random_state=42)

In [ ]:
Data